In [1]:
import glob
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import geopandas as gpd
import math
from shapely.geometry import Point

import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import dask_geopandas as dg

from pathlib import Path
import os
import glob

import re
from tqdm.notebook import tqdm

%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pbar = ProgressBar()
pbar.register()

# Input data

### sa data

In [2]:
%%time
sa = gpd.read_file('data/shapefile/Finn_SA2Summary.shp')
sa = sa[['BldgFloo_1', 'PopDensity', 'NumberBldg', 'geometry']] # drop column not used in caluclation of Ignition
sa = gpd.GeoDataFrame(sa)
sa_dask = dg.from_geopandas(sa, npartitions=4)

del sa

CPU times: user 145 ms, sys: 17.6 ms, total: 162 ms
Wall time: 264 ms


### pga data

In [3]:
%%time
pga = pd.read_parquet('/Users/alex/Dropbox/Work/Repository/OpenQuake/output/PGAxy.parquet')
pgaGeo = gpd.GeoDataFrame(pga, geometry=gpd.points_from_xy(pga.lon, pga.lat, crs="EPSG:2193"))
# pga_dask = dg.from_geopandas(pga, npartitions=4)


CPU times: user 3min 24s, sys: 13.9 s, total: 3min 38s
Wall time: 3min 59s


In [21]:
pgaGeo.plot()

AttributeError: 'GeometryArray' object has no attribute 'plot'

# Merge SA data with PGA events (dask-geodataframe)

In [17]:
type(pga_dask); type(sa_dask)

dask.dataframe.core.DataFrame

dask_geopandas.core.GeoDataFrame

In [ ]:
%%time
from spatialpandas import sjoin
pga_with_saData = sjoin(pga_dask, sa_dask)
# pga_with_saData = dg.sjoin(pgaGeo, sa_dask, op="within")

In [ ]:
%%time
PGA_SA = dg.sjoin(pga_dask, sa_dask)#, op="within")

In [ ]:
PGA_SA.to_parquet('/Users/alex/Dropbox/Work/Repository/OpenQuake/PGA_SA.parquet')